In [ ]:
#⚠️⚠️CAUTION: This code needs to run only once to setup the Results file

import pandas as pd
import numpy as np

df = pd.read_excel('./New_results.xlsx') # Change file name to Old_results.xlsx for old abstracts
df = df.dropna(axis = 0, ignore_index=True)
df['Sapling'] = ''
df.to_excel("Results_New.xlsx") # Change file name to Results_Old.xlsx for old abstracts results

In [ ]:
# function
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import time
import re

driver = webdriver.Chrome()
driver.get("https://sapling.ai/ai-content-detector")
driver.maximize_window()

elem = driver.find_element(By.ID, "content-editor")
error = driver.find_element(By.ID  , 'p-error')
df = pd.read_excel('./Results.xlsx')
df = df.dropna(axis = 0 , ignore_index=False , subset= 'Abstract')

# class for custom condition for Webdriverwait
class text_matches_float_regex:
    def __init__(self, locator):
        self.locator = locator
        self.pattern = re.compile(r'^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$')

    def __call__(self, driver):
        element = driver.find_element(*self.locator)

        if self.pattern.match(element.text) is not None :
            return element
        else:
            return False

# function for removing html tags
def remove_html_tags(text):
    clean = re.compile('<.*?>')  # Compile the regex pattern for HTML tags
     # Remove HTML tags
    text = re.sub(clean, '', text)
    # Replace underscores with empty spaces
    text = text.replace('_', ' ')
    return text

df['Abstract'] = df["Abstract"].apply(remove_html_tags)

# Main scrapping function
def ai_text_score(text):
    elem.clear()
    elem.send_keys(text)
    time.sleep(5)
    if error.is_displayed(): # id error occurs recall the function
        return ai_text_score(text)
    res = WebDriverWait(driver, 10).until(text_matches_float_regex((By.ID, "fake-prob")))
    return res.text

last_index = 910  #last index till the loop will run


try:
    for ind in df.index:
        if pd.isnull(df['Sapling'][ind]):
            df['Sapling'][ind]= ai_text_score(df['Abstract'][ind])
        if ind == last_index:
            break
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
     driver.close()
df.to_excel("Results.xlsx", index = False)
print(df)